# Batch Deep Learning Training for All Brazilian State Capitals (Stacked GRU)

This notebook trains the best deep learning model configuration (Stacked GRU, as determined in previous experiments) for each Brazilian state capital. Results are saved in a structured way for further analysis and comparison.

In [ ]:
import os
import pandas as pd
import numpy as np
from src.preprocessing import load_city_data, filter_city, clean_timeseries, prepare_data_for_model
from src.models import build_stacked_gru  # Use the best model function as determined
from src.train import train_model, evaluate_model, save_predictions, save_metrics
from src.utils import plot_training_history
from tqdm import tqdm

# --- List of state capitals and their IBGE codes ---
capitals = [
    ("Rio Branco", 1200401), ("Maceió", 2704302), ("Macapá", 1600303), ("Manaus", 1302603),
    ("Salvador", 2927408), ("Fortaleza", 2304400), ("Brasília", 5300108), ("Vitória", 3205309),
    ("Goiânia", 5208707), ("São Luís", 2111300), ("Cuiabá", 5103403), ("Campo Grande", 5002704),
    ("Belo Horizonte", 3106200), ("Belém", 1501402), ("João Pessoa", 2507507), ("Curitiba", 4106902),
    ("Recife", 2611606), ("Teresina", 2211001), ("Rio de Janeiro", 3304557), ("Natal", 2408102),
    ("Porto Alegre", 4314902), ("Porto Velho", 1100205), ("Boa Vista", 1400100), ("Florianópolis", 4205407),
    ("São Paulo", 3550308), ("Aracaju", 2800308), ("Palmas", 1721000)
]

# --- Model and data parameters (update as needed) ---
model_params = {
    'sequence_length': 12,
    'forecast_horizon': 4,
    'normalization': 'zscore',
    'val_size': 12
}
target_column = 'target'
results_dir = '../results/dl_batch_state_capitals'
os.makedirs(results_dir, exist_ok=True)

# --- Load data ---
data_path = '../data/df_base_morb_resp.csv'
df = load_city_data(data_path)

# --- Batch training ---
for city_name, cd_mun in tqdm(capitals, desc='State Capitals'):
    df_city = filter_city(df, cd_mun=cd_mun)
    df_city = clean_timeseries(df_city, target_column=target_column)
    data_dict = prepare_data_for_model(
        df=df_city,
        target_column=target_column,
        sequence_length=model_params['sequence_length'],
        forecast_horizon=model_params['forecast_horizon'],
        normalization=model_params['normalization'],
        val_size=model_params.get('val_size', None)
    )
    X_train = data_dict['X_train']
    y_train = data_dict['y_train']
    X_val = data_dict.get('X_val', None)
    y_val = data_dict.get('y_val', None)
    X_test = data_dict['X_test']
    y_test = data_dict['y_test']
    test_df = data_dict['test_df']
    scaler = data_dict.get('scaler')
    # Build and train model
    input_shape = X_train.shape[1:]
    model = build_stacked_gru(input_shape=input_shape, loss='huber')
    history = train_model(
        model=model,
        X_train=X_train,
        y_train=y_train,
        X_val=X_val,
        y_val=y_val,
        batch_size=32,
        epochs=100,
        patience=10,
        verbose=0
    )
    # Evaluate and save
    y_pred = model.predict(X_test)
    test_dates = test_df['week'].values[-len(y_test):] if 'week' in test_df.columns else np.arange(len(y_test))
    preds_file = save_predictions(
        y_true=y_test,
        y_pred=y_pred,
        dates=test_dates,
        city_name=city_name,
        model_name='gru_stacked',
        output_dir=results_dir
    )
    metrics = evaluate_model(model, X_test, y_test, scaler=scaler)
    metrics_file = save_metrics(
        metrics=metrics,
        city_name=city_name,
        model_name='gru_stacked',
        output_dir=results_dir,
        params=model_params
    )
    # Optionally, save training history plot
    if history is not None:
        fig = plot_training_history(history)
        fig.savefig(os.path.join(results_dir, f'{city_name}_gru_stacked_training_history.png'))

All predictions, metrics, and training history plots are saved in the results/dl_batch_state_capitals directory, one file per capital. You can rerun this notebook for other targets or model configs as needed.